# Self study 10

## Libraries
For this task you will need the SPARQLWrapper and pandas libraries. You can find the documentation of SPARQLWrapper [here](https://sparqlwrapper.readthedocs.io/en/latest/). After installing it, you can execute the following command.

In [6]:
from SPARQLWrapper import SPARQLWrapper, JSON
from pandas import json_normalize

## Introduction
In this session, you are expected to compose and submit queries to [Wikidata](https://www.wikidata.org/). Wikidata is one of the largest open knowledge graphs on the web, with more than 100 million nodes. Wikidata is built in a community effort: anyone can edit and contribute to it. You can find an introduction to Wikidata [here](https://www.wikidata.org/wiki/Wikidata:Introduction).

Wikidata uses the following convention for the resource names:
* The resources are identified by Qx, where x is a number
* The predicates are identified by Px, where x is a number

For example, Q601956 identifies Aalborg University, and an HTML representation of the resources is available [here](https://www.wikidata.org/wiki/Q601956).

To submit a SPARQL query, we need first to create a SPARQLWrapper object representing the SPARQL endpoint, i.e. the web service exposing an interface compatible with the SPARQL protocol, and which we can use to submit queries.

In [2]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

Next, we can create the SPARQL query to retrieve all the Danish (Q35) universities (Q3918). The result of the query is formatted in JSON. The Wikidata endpoint implicitly defines some prefixes, such as:
* wd for http://www.wikidata.org/entity/ (the namespace of the entities)
* wdt for http://www.wikidata.org/prop/direct/ (the namespace of the predicates)

This simplifies the query writing, as one can use the implicitly defined prefixes. 

In [3]:
sparql.setQuery("""
    SELECT ?uni
    WHERE {
        ?uni wdt:P31 wd:Q3918 .
        ?uni wdt:P17 wd:Q35 .
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results)

{'head': {'vars': ['uni']}, 'results': {'bindings': [{'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q124882'}}, {'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1542386'}}, {'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2166335'}}, {'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2267483'}}, {'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q4563603'}}, {'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q4583570'}}, {'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q4661370'}}, {'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q4662636'}}, {'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q601956'}}, {'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3359362'}}, {'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3436286'}}, {'uni': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3625669'}}, {'uni': {'typ

We can format the query result in a data frame to improve the readability.

In [4]:
results_df = json_normalize(results['results']['bindings'])
results_df[['uni.value']]

,uni.value
0,http://www.wikidata.org/entity/Q124882
1,http://www.wikidata.org/entity/Q1542386
2,http://www.wikidata.org/entity/Q2166335
3,http://www.wikidata.org/entity/Q2267483
4,http://www.wikidata.org/entity/Q4563603
5,http://www.wikidata.org/entity/Q4583570
6,http://www.wikidata.org/entity/Q4661370
7,http://www.wikidata.org/entity/Q4662636
8,http://www.wikidata.org/entity/Q601956
9,http://www.wikidata.org/entity/Q3359362


To further improve the readability, we may want to retrieve not only the IRI, but also a literal value acting as the "human-readable label" of the resource. In Wikidata, such labels are stored in a separate service. One can access to them using the federated SPARQL operator SERVICE, as follows. The Wikidata implementation of this service is customised, as it automatically generates a variable with the literal value, despite the variable name is not mentioned in the WHERE clause.

In [5]:
sparql.setQuery("""
    SELECT ?uni ?uniLabel
    WHERE {
        ?uni wdt:P31 wd:Q3918 .
        ?uni wdt:P17 wd:Q35 .
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results_df = json_normalize(results['results']['bindings'])
results_df[['uni.value', 'uniLabel.value']]

,uni.value,uniLabel.value
0,http://www.wikidata.org/entity/Q124882,IT University of Copenhagen
1,http://www.wikidata.org/entity/Q1542386,Roskilde University
2,http://www.wikidata.org/entity/Q601956,Aalborg University
3,http://www.wikidata.org/entity/Q4563603,Kunstakademiets Arkitektskole
4,http://www.wikidata.org/entity/Q4583570,Department of Information Studies
5,http://www.wikidata.org/entity/Q4661370,Aalborg Business College
6,http://www.wikidata.org/entity/Q4662636,DTU Diplom
7,http://www.wikidata.org/entity/Q2166335,University of Southern Denmark
8,http://www.wikidata.org/entity/Q2267483,Selandia - Center for Business Education
9,http://www.wikidata.org/entity/Q3359362,Danish Design School


You can find more information about querying Wikidata [here](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial). 

## Task
Write SPARQL queries for the Wikidata endpoint to answer the following questions:
* Who is employed at Aalborg University?
* What are the top 5 most popular occupations of Aalborg University employees?
* For each Danish university, find the number of employees.
* Retrieve ten Scandinavian cities, ordered by population.
* Find the ten Danish cities with the highest number of twinned cities.
* How many Danish people have a node in Wikidata?

Hints:
* Navigate Wikidata to find the resource and predicate codes. 
* There is a list of all the properties in Wikidata [here](https://www.wikidata.org/wiki/Wikidata:List_of_properties/all_in_one_table)

In [30]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

def doQuery(setQuery, displayColumns=[]):
    sparql.setQuery(setQuery)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = json_normalize(results['results']['bindings'])
    display(results_df[displayColumns])

#Who is employed at Aalborg University?
# ?person wdt:employer wd:Aalborg University
doQuery(setQuery="""
    SELECT ?person ?personLabel
    WHERE {
        ?person wdt:P108 wd:Q601956
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
""", displayColumns=['person.value', 'personLabel.value'])


#What are the top 5 most popular occupations of Aalborg University employees?
doQuery(setQuery="""
    SELECT ?person ?personLabel
    WHERE {
        ?person wdt:P108 wd:Q601956 
        ?person wdt:P108 wd:Q124882
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
""")



,person.value,personLabel.value
0,http://www.wikidata.org/entity/Q121253,Mike Sandbothe
1,http://www.wikidata.org/entity/Q935036,Louis Becker
2,http://www.wikidata.org/entity/Q1395405,Lars Graugaard
3,http://www.wikidata.org/entity/Q1532363,Christian Graugaard
4,http://www.wikidata.org/entity/Q2443716,Torben Brostrøm
...,...,...
940,http://www.wikidata.org/entity/Q118038571,Evangelos Boukas
941,http://www.wikidata.org/entity/Q119499075,Jacobo Rouces
942,http://www.wikidata.org/entity/Q119983834,Sandra Cassotta
943,http://www.wikidata.org/entity/Q120403013,Kerli Kant-Hvass


QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'SPARQL-QUERY: queryStr=\n    SELECT ?person ?personLabel\n    WHERE {\n        ?person wdt:P108 wd:Q601956 \n        ?person wdt:P108 wd:Q124882\n        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }\n    }\n\njava.util.concurrent.ExecutionException: org.openrdf.query.MalformedQueryException: Encountered " <VAR1> "?person "" at line 5, column 9.\nWas expecting one of:\n    "{" ...\n    "}" ...\n    ";" ...\n    "," ...\n    "." ...\n    "optional" ...\n    "graph" ...\n    "minus" ...\n    "filter" ...\n    "bind" ...\n    "service" ...\n    "values" ...\n    "include" ...\n    "let" ...\n    "." ...\n    \n\tat java.util.concurrent.FutureTask.report(FutureTask.java:122)\n\tat java.util.concurrent.FutureTask.get(FutureTask.java:206)\n\tat com.bigdata.rdf.sail.webapp.BigdataServlet.submitApiTask(BigdataServlet.java:292)\n\tat com.bigd